# Модуль 1 Парсинг и предобработка данных

In [230]:
#импорт библиотек
import glob
import os
import pandas as pd
import numpy as np
import json
import docx
from bs4 import BeautifulSoup as bs
import requests
from tqdm.auto import tqdm, trange
from pymystem3 import Mystem
from nltk.stem.snowball import SnowballStemmer 
from nltk.tokenize import word_tokenize
from nltk.stem import *
from nltk.corpus import stopwords
import string
import re
import pyLDAvis.sklearn
import pyLDAvis
import pyLDAvis.lda_model
import pymorphy2
import warnings
warnings.filterwarnings("ignore")
import nltk
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from gensim.models import Word2Vec
import pylab
import scipy.stats as stats
from sklearn import preprocessing
import seaborn as sns

# 1.1 Парсинг данных

## 1.1.1 Париснг json файлов

In [125]:
alljson = glob.glob("*.json")

In [126]:
alljson

['Cybersport.Metaratings.json',
 'ESforce Holding.json',
 'Gett.json',
 'Goose Gaming.json',
 'MMA.Metaratings.json',
 'skillbox.json',
 'Vinci Agency.json',
 'Гэллэри Сервис.json',
 'Здоровье.ру.json',
 'Иннотех.json',
 'Проект по использованию технологий компьютерного зрения на базе искусственного интеллекта для анализа медицинских изображений.json',
 'Промобот.json',
 'Студия Артемия Лебедева.json',
 'Федерация креативных индустрий.json']

С помощью библиотеки glob создал список всех файлов формата json

In [127]:
result_list = {'namecompany': [], 'text': [], 'ratingarticle': [], 'date': [], 'rating': [], 'industries': [], 'about': []}

Создан словарь для заполнения его набором данных

In [128]:
for i in range(len(alljson)):
    with open(alljson[i], "r", encoding="utf-8") as file:
        articles_json = file.read()
    articles = json.loads(articles_json)
    if(articles['info'] != None): #если в json файле содержится информация о компании
        for k in range(len(articles['info'])):
            n = articles['info']
            result_list['rating'].append(n['rate']) #добавление рейтинга компании
            result_list['industries'].append(n['industries']) #добавление отраслей компании
            result_list['about'].append(n['about']) #добавление описания компании
        for v in range(len(articles['refs']) - len(articles['info'])): #заполнение осатвшихся статей
            result_list['rating'].append(np.nan)
            result_list['industries'].append(np.nan)
            result_list['about'].append(np.nan)
    else: #если в json файле не содержится информации о компании
        for l in range(len(articles['refs'])):
            result_list['rating'].append(np.nan)
            result_list['industries'].append(np.nan)
            result_list['about'].append(np.nan)
    for j in range(len(articles['refs'])): #заполенение словаря информацией о статьи
            n = articles['refs'][j]
            if(n is not None):
                aa = alljson[i]
                spl = aa.split('.')
                for p in spl:
                    if p == 'json':
                        spl.remove(p)
                aa = '.'.join(spl)
                result_list['namecompany'].append(aa) #добавление названии компании
                result_list['text'].append(n[0]) #добавление текста статьи
                result_list['ratingarticle'].append(n[2]['rate']) #добавление рейтинга статьи
                result_list['date'].append(n[1]['day'] + ' ' + n[1]['month'] + ' ' + n[1]['time']) #добавление даты публикации статьи
            else:
                result_list['namecompany'].append(np.nan)
                result_list['text'].append(np.nan)
                result_list['ratingarticle'].append(np.nan)
                result_list['date'].append(np.nan)

In [129]:
df = pd.DataFrame(data=result_list)

In [130]:
df.head()

,namecompany,text,ratingarticle,date,rating,industries,about
0,Cybersport.Metaratings,7 декабря состоялась церемония награждения лау...,6.0,8 декабря 09:30,NaN,NaN,NaN
1,ESforce Holding,7 декабря состоялась церемония награждения лау...,6.0,8 декабря 09:30,NaN,NaN,NaN
2,ESforce Holding,Из совместного исследования издателя компьютер...,12.0,29 июля 13:50,NaN,NaN,NaN
3,ESforce Holding,По статистике The Esports Observer за I полови...,18.0,9 июля 09:33,NaN,NaN,NaN
4,ESforce Holding,Компании SuperData Research и PayPal провели с...,11.0,11 апреля 12:43,NaN,NaN,NaN


Получен набор данных из json файлов. Можно заметить, что во многих записей отсутствует информация о компании, а именно рейтинг компании, отрасли и описание.

## 1.1.2 Парсинг с сайта habr

In [131]:
content = docx.Document('Condidates.docx').paragraphs

In [132]:
companies = []
for i in range(len(content)):
    if(content[i].text != ''):
        companies.append(content[i].text)
companies

['Здоровье.ру',
 'Проект по использованию технологий компьютерного зрения на базе искусственного интеллекта (ИИ) для анализа медицинских изображений',
 'Skillbox ',
 'Gett',
 'Промобот',
 'Иннотех',
 'Cybersport.Metaratings',
 'MMA.Metaratings',
 'Goose Gaming',
 'ESforce Holding',
 'Vinci Agency',
 'Гэллэри Сервис',
 'Студия Артемия Лебедева',
 'Федерация креативных индустрий',
 'Метарейтинг',
 'СберМаркет',
 'Balance Platform',
 'Московская биржа',
 'Samsung Electronics',
 'Нетология',
 'Дневник МЭШ',
 'Цифровое образование']

Заполнил список компаний, которые участвуют в конкурсе из файла Condidates.docx

In [133]:
compnotjson = []
for i in range(len(companies)):
    if companies[i] + '.json' not in alljson:
        compnotjson.append(companies[i])
print(compnotjson)

['Проект по использованию технологий компьютерного зрения на базе искусственного интеллекта (ИИ) для анализа медицинских изображений', 'Skillbox ', 'Метарейтинг', 'СберМаркет', 'Balance Platform', 'Московская биржа', 'Samsung Electronics', 'Нетология', 'Дневник МЭШ', 'Цифровое образование']


Список компаний, по которым нет информации в json файлах

In [134]:
from datetime import date
datelimit = date(2022, 1, 1)

In [135]:
datelimit.year

2022

In [136]:
from datetime import date
for i in tqdm(range(len(compnotjson))):
    url = 'https://habr.com/ru/search/?q=' + compnotjson[i] + '&target_type=companies&order=relevance' #переход на страницу поиска компании по названию
    page = requests.get(url)
    soup = bs(page.text, 'html.parser')
    c = soup.find('div', class_ = 'tm-company-snippet__info')
    if(c is not None):
        url = 'https://habr.com' + c.a.get('href') #переход на страницу профиля компании
        page = requests.get(url)
        soup = bs(page.text, 'html.parser')
        #получение рейтинга компании
        rate = soup.find('span', class_='tm-votes-lever__score-counter tm-votes-lever__score-counter tm-votes-lever__score-counter_rating')
        rate = ''.join(rate.text.split())
        #получение описания компании
        about = soup.find('div', class_='tm-company-card__description')
        about = about.text
        #получение отраслей компании
        fieldtext = "" 
        field = soup.find_all('a', class_='tm-company-profile__categories-text') #получаем список отраслей компании
        for fields in field:
            fieldtext = fieldtext + ((fields.text).strip()) + " "
        fieldtext = fieldtext.splitlines()
        #получение названии комании
        namecompany = soup.find('a', class_='tm-company-card__name router-link-exact-active router-link-active')
        if(namecompany.text in compnotjson):
            #переход на страницу поиска статей по названию компании
            url = 'https://habr.com/ru/search/?q=' + compnotjson[i] + '&target_type=posts&order=relevance'
            page = requests.get(url)
            soup = bs(page.text, 'html.parser')
            #получение максимального количества страниц по заданному поиску
            pages = soup.find_all('a', class_='tm-pagination__page')
            pages = int(pages[1].text)
        
        
            for k in tqdm(range(pages)):
                #переход по страницам
                url = 'https://habr.com/ru/search/page' + str(k) + '/?q=' + compnotjson[i] + '&target_type=posts&order=relevance'
                page = requests.get(url)
                soup = bs(page.text, 'html.parser')
                #получение всех заголовков статей
                articles = soup.find_all('h2', class_='tm-title tm-title_h2')
                for j in tqdm(articles):
                    url = 'https://habr.com' + j.a.get('href') #переход на страницу статьи
                    page = requests.get(url)
                    soup = bs(page.text, 'html.parser')
                    #получение даты публикации статьи
                    datepub = soup.find('span', class_= 'tm-article-datetime-published')
                    
                    date2 = datepub.time.get('title').split('-')
                    date3 = date2[2].split(',')
                    
                    year = int(date2[0])
                    month = int(date2[1])
                    day = int(date3[0])
                    
                    if(year <= datelimit.year): #если статья была выпущена до 2022 года
                        #получение текста статьи
                        textarticle = soup.find('div', class_='tm-article-body')
                        if(textarticle is not None):
                            texttt = textarticle.text
                        else:
                            texttt = np.nan
                        result_list['text'].append(texttt)
                        if(datepub is not None):
                            date = datepub.text.split()
                        
                            month = date[1]
                            if(month == "сен"):
                                month = "сентября"
                            if(month == "окт"):
                                month = "октября"
                            if(month == "ноя"):
                                month = "ноября"
                            if(month == "дек"):
                                month = "декабря"
                            if(month == "янв"):
                                month = "января"
                            if(month == "фев"):
                                month = "февраля"
                            if(month == "апр"):
                                month = "апреля"
                            if(month == "май"):
                                month = "мая"
                            if(month == "июн"):
                                month = "июня"
                            if(month == "июл"):
                                month = "июля"
                            if(month == "авг"):
                                month = "августа"
                            if(month == "мар"):
                                month = "марта"
                            if(len(date) == 5):
                                datetext = date[0] + " " + month + " " + date[4]
                            elif(len(date) == 4):
                                datetext = date[0] + " " + month + " " + date[3]
                        
                            result_list['date'].append(datetext)
                        
                        else:
                            result_list['date'].append(np.nan)
                        #заполнение словаря названием компаний
                        result_list['namecompany'].append(namecompany.text)
                        #заполнение словаря рейтингом статьи
                        result_list['ratingarticle'].append(np.nan)
                        #заполнение словаря рейтингом компании
                        result_list['rating'].append(rate)
                        #заполнение словаря описанием компании
                        result_list['about'].append(about)
                        #заполнение словаря отраслями компании
                        result_list['industries'].append(''.join(str(e) for e in fieldtext))

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

0it [00:00, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

...

  0%|          | 0/19 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

In [142]:
df = pd.DataFrame(data=result_list)
df.to_csv('dataset.csv', index=False)

In [143]:
df.head()

,namecompany,text,ratingarticle,date,rating,industries,about
0,Cybersport.Metaratings,7 декабря состоялась церемония награждения лау...,6.0,8 декабря 09:30,NaN,NaN,NaN
1,ESforce Holding,7 декабря состоялась церемония награждения лау...,6.0,8 декабря 09:30,NaN,NaN,NaN
2,ESforce Holding,Из совместного исследования издателя компьютер...,12.0,29 июля 13:50,NaN,NaN,NaN
3,ESforce Holding,По статистике The Esports Observer за I полови...,18.0,9 июля 09:33,NaN,NaN,NaN
4,ESforce Holding,Компании SuperData Research и PayPal провели с...,11.0,11 апреля 12:43,NaN,NaN,NaN


## 1.1.3 Описание набора данных


__namecompany__ - название компании.

__about__ - описание компании.

__industries__ - сферы деятельности компаний.

__date__ - дата публикации статьи.

__text__ - текст статьи.

__ratingarticle__ - рейтинг статьи.

__rating__ - рейтинг компании.

In [144]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1627 entries, 0 to 1626
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   namecompany    1623 non-null   object 
 1   text           1623 non-null   object 
 2   ratingarticle  1112 non-null   float64
 3   date           1623 non-null   object 
 4   rating         519 non-null    object 
 5   industries     519 non-null    object 
 6   about          519 non-null    object 
dtypes: float64(1), object(6)
memory usage: 89.1+ KB


В результате был получен набор данных состоящий из 1627 записей, также в данном наборе содержатся записи с пустым значением текста статьи, поэтому в дальнейшем они будут удалены.

# 1.2 Формирование структуры набора данных

## 1.2.1 Удаление атрибутов 

Обоснование удаления атрибутов: в наборе данных есть признаки, которые не несут никакой важной информации -'ratingarticle','about'. Первый признак является нумерацией строк в датафрейме, который уже есть в нем.

In [145]:
df = df.drop(["ratingarticle", "about"], axis=1) #Удаление атрибутов

В данном наборе данных я оставил такие атрибуты как: название компании, текст статьи, рейтинг компании, дата публикации статьи и сферы дейятельности компании. Название компании требуется для того, чтобы знать, какая компания номинируется на определенную номинацию. Текст статьи требуется для обучения и прогнозирования номинации. Рейтинг компании и дата публикации статьи требуются для графиков зависимости.

## 1.2.2 Удаление строк

In [146]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1627 entries, 0 to 1626
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   namecompany  1623 non-null   object
 1   text         1623 non-null   object
 2   date         1623 non-null   object
 3   rating       519 non-null    object
 4   industries   519 non-null    object
dtypes: object(5)
memory usage: 63.7+ KB


Можно заметить, что существуют записи, в которых все значения пусты. Поэтому эти записи нужно удалить

In [147]:
df=df[df['text'].notnull()]

In [148]:
df.isnull().sum()

namecompany       0
text              0
date              0
rating         1104
industries     1104
dtype: int64

In [149]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1623 entries, 0 to 1626
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   namecompany  1623 non-null   object
 1   text         1623 non-null   object
 2   date         1623 non-null   object
 3   rating       519 non-null    object
 4   industries   519 non-null    object
dtypes: object(5)
memory usage: 76.1+ KB


Большое количество пустых значений содержатся в записях, полученных из json файлов.

В итоге были удалены статьи у которых все значения пусты, но все еще остались атрибуты rating и industries, которые тоже пусты.

## 1.2.3 Обработка пустых данных

In [154]:
df = df.astype({'rating': np.float})

In [155]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1623 entries, 0 to 1626
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   namecompany  1623 non-null   object 
 1   text         1623 non-null   object 
 2   date         1623 non-null   object 
 3   rating       519 non-null    float64
 4   industries   519 non-null    object 
dtypes: float64(1), object(4)
memory usage: 76.1+ KB


In [156]:
df['rating'] = df['rating'].fillna(df['rating'].mean())

In [157]:
df['industries'] = df['industries'].fillna(df['namecompany'])

Замена пустых рейтингов средними значениями, а также пустых сфер деятельности компаний названиями компаний.

In [160]:
df.head()

,namecompany,text,date,rating,industries
0,Cybersport.Metaratings,7 декабря состоялась церемония награждения лау...,8 декабря 09:30,74.799364,Cybersport.Metaratings
1,ESforce Holding,7 декабря состоялась церемония награждения лау...,8 декабря 09:30,74.799364,ESforce Holding
2,ESforce Holding,Из совместного исследования издателя компьютер...,29 июля 13:50,74.799364,ESforce Holding
3,ESforce Holding,По статистике The Esports Observer за I полови...,9 июля 09:33,74.799364,ESforce Holding
4,ESforce Holding,Компании SuperData Research и PayPal провели с...,11 апреля 12:43,74.799364,ESforce Holding


In [161]:
df.isnull().sum()

namecompany    0
text           0
date           0
rating         0
industries     0
dtype: int64

In [162]:
df.to_csv('dataset.csv', index=False)

В итоге получен набор данных без пропусков данных и пустых значений

# 1.3 Предварительная обработка текстовых данных

## 1.3.1 Удаление знаков препинания, пробелов и чисел 

In [163]:
def remove_punctuation(text): #Функция удаления знаков препинания
    return "".join([ch if ch not in string.punctuation else ' ' for ch in text])
def remove_numbers(text): #Функция удаления чисел
    return ''.join([i if not i.isdigit() else ' ' for i in text])
def remove_multiple_spaces(text): #Функция удаления пробелов
    return re.sub(r'\s+', ' ', text, flags=re.I)

С помощью string.punctuation можно получить все знаки препинания и легко их удалить. Использую регулярное выражение для поиска пробелов и их удаления с помощью функции re.sub().

In [164]:
prep_text = [remove_multiple_spaces(remove_numbers(remove_punctuation(text.lower()))) for text in tqdm(df['text'])]

  0%|          | 0/1623 [00:00<?, ?it/s]

In [165]:
df['text_prep'] = prep_text

In [166]:
df.head()

,namecompany,text,date,rating,industries,text_prep
0,Cybersport.Metaratings,7 декабря состоялась церемония награждения лау...,8 декабря 09:30,74.799364,Cybersport.Metaratings,декабря состоялась церемония награждения лаур...
1,ESforce Holding,7 декабря состоялась церемония награждения лау...,8 декабря 09:30,74.799364,ESforce Holding,декабря состоялась церемония награждения лаур...
2,ESforce Holding,Из совместного исследования издателя компьютер...,29 июля 13:50,74.799364,ESforce Holding,из совместного исследования издателя компьютер...
3,ESforce Holding,По статистике The Esports Observer за I полови...,9 июля 09:33,74.799364,ESforce Holding,по статистике the esports observer за i полови...
4,ESforce Holding,Компании SuperData Research и PayPal провели с...,11 апреля 12:43,74.799364,ESforce Holding,компании superdata research и paypal провели с...


Удалены знаки препинания, числа и пробелы.

## 1.3.2 Удаление стоп-слов

В билиотеке NLTK есть встроенный список стоп-слов, который можно использовать, чтобы удалить стоп-слова из текста. Однако это не универсальный список стоп-слов для любой задачи, так что нужно будет его дополнить.

Также в список стоп слов добавлю спецсимволы для их удаления из текста.

In [170]:
russian_stopwords = stopwords.words("russian") #создание списка стопслов
russian_stopwords.extend(['…', '«', '»', '...', 'т.д.', 'т', 'д', 'это','—', '“', '”', 'который', '„']) #Дополнение стопслов

In [171]:
sw_texts_list = [] #создание списка текста с удаленными стоп словами
for text in tqdm(df['text_prep']):
    tokens = word_tokenize(text)    
    tokens = [token for token in tokens if token not in russian_stopwords and token != ' ']
    text = " ".join(tokens)
    sw_texts_list.append(text)

df['text_sw'] = sw_texts_list

  0%|          | 0/1623 [00:00<?, ?it/s]

Для дополнения стоп слов воспользуюсь токенизацией.

In [172]:
word_tokenize(df['text_sw'][0])

['декабря',
 'состоялась',
 'церемония',
 'награждения',
 'лауреатов',
 'премии',
 'рунета',
 'года',
 'году',
 'премию',
 'подано',
 'работ',
 'вклад',
 'развитие',
 'российского',
 'сегмента',
 'сети',
 'интернет',
 'девять',
 'основных',
 'четыре',
 'специальные',
 'номинации',
 'финалистами',
 'конкурса',
 'каждой',
 'номинации',
 'стали',
 'организаций',
 'лауреатами',
 'премии',
 'рунета',
 'стали',
 'компании',
 'персоны',
 'причем',
 'раньше',
 'организаторы',
 'российская',
 'ассоциация',
 'электронных',
 'коммуникаций',
 'раэк',
 'награждали',
 'несколько',
 'лучших',
 'представителей',
 'каждой',
 'номинации',
 'мнению',
 'экспертного',
 'сообщества',
 'году',
 'обратились',
 'помощи',
 'искусственного',
 'интеллекта',
 'добавили',
 'одного',
 'лауреата',
 'выбирался',
 'именно',
 'мнению',
 'ии',
 'номинация',
 'государство',
 'общество',
 'победители',
 'благотворительный',
 'фонд',
 'подари',
 'жизнь',
 'ано',
 'диалог',
 'правительство',
 'москвы',
 'проект',
 'цифровая'

In [173]:
df.head()

,namecompany,text,date,rating,industries,text_prep,text_sw
0,Cybersport.Metaratings,7 декабря состоялась церемония награждения лау...,8 декабря 09:30,74.799364,Cybersport.Metaratings,декабря состоялась церемония награждения лаур...,декабря состоялась церемония награждения лауре...
1,ESforce Holding,7 декабря состоялась церемония награждения лау...,8 декабря 09:30,74.799364,ESforce Holding,декабря состоялась церемония награждения лаур...,декабря состоялась церемония награждения лауре...
2,ESforce Holding,Из совместного исследования издателя компьютер...,29 июля 13:50,74.799364,ESforce Holding,из совместного исследования издателя компьютер...,совместного исследования издателя компьютерных...
3,ESforce Holding,По статистике The Esports Observer за I полови...,9 июля 09:33,74.799364,ESforce Holding,по статистике the esports observer за i полови...,статистике the esports observer i половину год...
4,ESforce Holding,Компании SuperData Research и PayPal провели с...,11 апреля 12:43,74.799364,ESforce Holding,компании superdata research и paypal провели с...,компании superdata research paypal провели сов...


In [174]:
df.drop(['text_prep'], axis=1, inplace=True)

В итоге получил текста статей без стоп слов, что в дальнейшем пригодится для лемматизации.

## 1.3.3 Лемматизация

В сравнение двух библиотек PyMystem3 от Яндекса и PyMorphy2, лучшим вариантом для лемматизации стала библиотека PyMorphy2, так как она выполнила лемматизацию в два раза быстрее чем PyMystem3

In [176]:
morph = pymorphy2.MorphAnalyzer(lang='ru')
lemm_texts_list = []
for text in tqdm(df['text_sw']):
    try:
        tokens = word_tokenize(text)
        res = list()
        for word in tokens:
            p = morph.parse(word)[0]
            res.append(p.normal_form)
        text = " ".join(res)
        lemm_texts_list.append(text)
    except Exception as e:
        print(e)
    
df['text_lemm'] = lemm_texts_list

  0%|          | 0/1623 [00:00<?, ?it/s]

In [177]:
df.head()

,namecompany,text,date,rating,industries,text_sw,text_lemm
0,Cybersport.Metaratings,7 декабря состоялась церемония награждения лау...,8 декабря 09:30,74.799364,Cybersport.Metaratings,декабря состоялась церемония награждения лауре...,декабрь состояться церемония награждение лауре...
1,ESforce Holding,7 декабря состоялась церемония награждения лау...,8 декабря 09:30,74.799364,ESforce Holding,декабря состоялась церемония награждения лауре...,декабрь состояться церемония награждение лауре...
2,ESforce Holding,Из совместного исследования издателя компьютер...,29 июля 13:50,74.799364,ESforce Holding,совместного исследования издателя компьютерных...,совместный исследование издатель компьютерный ...
3,ESforce Holding,По статистике The Esports Observer за I полови...,9 июля 09:33,74.799364,ESforce Holding,статистике the esports observer i половину год...,статистика the esports observer i половина год...
4,ESforce Holding,Компании SuperData Research и PayPal провели с...,11 апреля 12:43,74.799364,ESforce Holding,компании superdata research paypal провели сов...,компания superdata research paypal провести со...


## 1.3.4 Выделение частей речи

Процесс преобразования предложения в формы, то есть в список слов или кортежей (каждый кортеж имеет вид [слово, часть речи]).

pymorphy2 умеет разбирать не только словарные слова; для несловарных слов автоматически задействуется предсказатель.

In [178]:
ps_text_list = []
for text in tqdm(df['text_lemm']):
    try:
        words = word_tokenize(text)
        text_list = []
        for i in range(len(words)):
            p = morph.parse(words[i])[0]
            text_list.append("(" + words[i] + ", " + str(p.tag.POS) + ")")
        ps_text_list.append(text_list)
    except Exception as e:
        print(e)
df['text_ps'] = ps_text_list

  0%|          | 0/1623 [00:00<?, ?it/s]

In [179]:
df.head()

,namecompany,text,date,rating,industries,text_sw,text_lemm,text_ps
0,Cybersport.Metaratings,7 декабря состоялась церемония награждения лау...,8 декабря 09:30,74.799364,Cybersport.Metaratings,декабря состоялась церемония награждения лауре...,декабрь состояться церемония награждение лауре...,"[(декабрь, NOUN), (состояться, INFN), (церемон..."
1,ESforce Holding,7 декабря состоялась церемония награждения лау...,8 декабря 09:30,74.799364,ESforce Holding,декабря состоялась церемония награждения лауре...,декабрь состояться церемония награждение лауре...,"[(декабрь, NOUN), (состояться, INFN), (церемон..."
2,ESforce Holding,Из совместного исследования издателя компьютер...,29 июля 13:50,74.799364,ESforce Holding,совместного исследования издателя компьютерных...,совместный исследование издатель компьютерный ...,"[(совместный, ADJF), (исследование, NOUN), (из..."
3,ESforce Holding,По статистике The Esports Observer за I полови...,9 июля 09:33,74.799364,ESforce Holding,статистике the esports observer i половину год...,статистика the esports observer i половина год...,"[(статистика, NOUN), (the, None), (esports, No..."
4,ESforce Holding,Компании SuperData Research и PayPal провели с...,11 апреля 12:43,74.799364,ESforce Holding,компании superdata research paypal провели сов...,компания superdata research paypal провести со...,"[(компания, NOUN), (superdata, None), (researc..."


In [181]:
df.to_csv('dataset.csv', index=False) #сохранение обработанных данных

# 1.6 Подготовка отчета

## 1.6.1 Парсинг данных

1. Получен список всех json файлов
2. Создан словарь для заполнения его данными
3. Получен набор данных с json файлов. Во многих записей отсутствовала информация о компании, а именно рейтинг компании, отрасли и описание.
4. Получен список компаний, которые учавствуют в конкурсе из файла Condidates.docx
5. Получен список названий компаний, у которых нет информации в json файлах
6. Получен набор данных с сайта habr
7. Объединены наборы данных с json файлов и сайта

В результате был получен набор данных состоящий из 1627 записей, также в данном наборе содержатся записи с пустым значением текста статьи, поэтому в дальнейшем они будут удалены.

## 1.6.2 Формирование структуры набора данных

1. Удалены атрибуты'ratingarticle','about'
2. Удалены статьи текст которых пустой
3. Рейтинги, которые пусты заменил на средние значения
4. Сферы деятельности, которые пусты заменил на названия компаний

В данном наборе данных остались такие атрибуты как: название компании, текст статьи, рейтинг компании, дата публикации статьи и сферы дейятельности компании. Название компании требуется для того, чтобы знать, какая компания номинируется на определенную номинацию. Текст статьи требуется для обучения и прогнозирования номинации. Рейтинг компании и дата публикации статьи требуются для графиков зависимости.

В итоге получен набор данных без пропусков данных и пустых значений.

## 1.6.3 Предварительная обработка текстовых данных

1. Удалены знаки препинания, числа и пробелы.
2. Создание и дополнение списка стоп-слов
3. Создание списка текста статей с удаленными стоп-словами и добавление его в набор данных
4. Получение лемматизированного списка и добавление его в набор данных
5. Выделение частей речи

В набор данных были добавлены предобработанные текстовые данные

## 1.6.4 Поиск ключевых слов/n-грамм, векторизация текстов

1. Выбраны алгоритмы векторизации
2. Выполнена векторизация мешка слов
3. Выполнена векторизация TF-IDF
4. Выполнена векторизация Word Embeddings
5. Добавление ключевых слов, биграмм и триграмм

В результате в набор данных были добавлены ключевые слова, биграммы и триграммы.

## 1.6.5 Разведочный анализ

1. Открытие json и создание датафрейма целевой переменной
2. Добавление целевой переменной в основной набор данных
3. Проведена графическая оценка распределения целевой переменной
4. Проведена статическая оценка распределения целевой переменной методом Тест Шапиро-Уилка
5. Проведена статическая оценка распределения целевой переменной методом К-квадрат Д’Агостино
6. Построен график зависимости темы от рейтинга
7. Построен график зависимости темы от месяца публикации
8. Визуализация зависимости темы от ключевых слов
9. Построен график количества публикаций по названиям компаний
10. Построен график количества публикаций по рейтингу

Графическая и статическая оценка распределения целевой переменной показала, что распределение переменной не является нормальным / гауссовым.